In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Define thresholds (for simplification)
FCR_THRESHOLD = 0.02  # Frequency deviation (e.g., 50.02 Hz) for FCR activation
AFRR_THRESHOLD = 0.05 # Frequency deviation (e.g., 50.05 Hz) for aFRR activation


def bess_reserve_response(current_frequency, reference_frequency, max_capacity, current_soc):
    """
    Simulates a simplified BESS response to grid frequency deviations 
    by providing FCR and aFRR services.
    
    Returns: The power output/input decision (Positive = Discharge, Negative = Charge)
    """

    MAX_C_RATE = 0.5 * max_capacity # Maximum power to charge/discharge (0.5C)
    
    frequency_deviation = current_frequency - reference_frequency
    power_output = 0 # Default is to hold

    # --- 1. FCR (Fast, Primary Response) ---
    if abs(frequency_deviation) > FCR_THRESHOLD:
        # Simple linear response proportional to deviation
        # Negative deviation (low frequency) -> Positive power needed (Discharge)
        power_needed = -frequency_deviation * 500  # Scaling factor for power
        power_output = max(min(power_needed, MAX_C_RATE), -MAX_C_RATE) # Clamp to max C-rate

    # --- 2. aFRR (Slower, Secondary Response - Activated only if FCR is not sufficient/active) ---
    elif abs(frequency_deviation) > AFRR_THRESHOLD:
        # If frequency is LOW (deficit -> need POSITIVE reserve -> DISCHARGE)
        if frequency_deviation < 0: 
            power_output = MAX_C_RATE * 0.5  # aFRR Positive
            
        # If frequency is HIGH (surplus -> need NEGATIVE reserve -> CHARGE)
        elif frequency_deviation > 0:
            power_output = -MAX_C_RATE * 0.5  # aFRR Negative

    # --- 3. BESS State of Charge Constraint ---
    # Constraints are simplified to prevent charging when full or discharging when empty
    if power_output > 0 and current_soc <= 0.1: # Trying to discharge but battery is too low
        power_output = 0
    elif power_output < 0 and current_soc >= 0.9: # Trying to charge but battery is too full
        power_output = 0

    return power_output


In [14]:
# --- Setup Simulation Data ---
MAX_CAPACITY = 100 # MWh
REF_FREQ = 50.00
INITIAL_SOC = 0.50 

# Simulate a 30-minute period (30 steps, 1-minute resolution)
time_steps = 45 
time = np.arange(0, 0.5,time_steps) # Time in minutes

In [16]:
# Create a challenging frequency signal with multiple events
frequency_signals = np.full(time_steps, REF_FREQ)
frequency_signals[5:10] = 49.97  # Event 1: Low Frequency (FCR positive response)
frequency_signals[12:18] = 50.04 # Event 2: High Frequency (FCR negative response)
frequency_signals[22:25] = 49.90 # Event 3: Significant Low Frequency (FCR and potential aFRR)

In [17]:
# --- Run Simulation and Record Results ---
soc_levels = [INITIAL_SOC]
power_outputs = []
current_soc = INITIAL_SOC

for i in range(time_steps):
    freq = frequency_signals[i]
    
    # Get BESS decision (Power is in MW, based on C-rate)
    power = bess_reserve_response(freq, REF_FREQ, MAX_CAPACITY, current_soc)
    power_outputs.append(power)
    
    # Update SOC (Energy change MWh = Power MW * Time hours)
    energy_change_mwh = power * (1/60) 
    soc_change = energy_change_mwh / MAX_CAPACITY 
    current_soc += soc_change
    
    # Clamp SOC between 0 and 1
    current_soc = np.clip(current_soc, 0, 1)
    soc_levels.append(current_soc)

In [ ]:
# --- Plotting Code ---

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# 1. Frequency and BESS Response Plot
ax1.plot(time, frequency_signals, label='Grid Frequency (Hz)', color='blue')
ax1.axhline(REF_FREQ, color='black', linestyle='--', label=f'Reference ({REF_FREQ} Hz)')
ax1.axhline(REF_FREQ + FCR_THRESHOLD, color='red', linestyle=':', label='FCR Threshold')
ax1.axhline(REF_FREQ - FCR_THRESHOLD, color='red', linestyle=':')
ax1.set_ylabel('Frequency (Hz)')
ax1.set_title('BESS Frequency Response Simulation (FCR/aFRR)')
ax1_twin = ax1.twinx()
ax1_twin.step(time, power_outputs, where='mid', color='green', linestyle='-', label='BESS Power Output (MW)')
ax1_twin.set_ylabel('BESS Power (MW) [Discharge (+)/Charge (-)]', color='green')
ax1_twin.tick_params(axis='y', labelcolor='green')
ax1.legend(loc='upper left')
ax1_twin.legend(loc='upper right')
ax1.grid(True)

# 2. State of Charge (SOC) Plot
# The SOC array has one extra point (initial state), so use up to time_steps for X-axis
ax2.plot(np.arange(0, time_steps + 1), soc_levels, marker='.', linestyle='-', color='purple')
ax2.set_ylabel('State of Charge (SOC) (%)')
ax2.set_xlabel('Time (Minutes)')
ax2.axhline(1.0, color='red', linestyle='--', label='100% (Full)')
ax2.axhline(0.0, color='red', linestyle=':', label='0% (Empty)')
ax2.set_yticks(np.arange(0, 1.1, 0.2))
ax2.set_yticklabels([f'{int(s*100)}%' for s in np.arange(0, 1.1, 0.2)])
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig('bess_frequency_response_plots.png')

ValueError: x and y must have same first dimension, but have shapes (1,) and (45,)